In [ ]:
!pip install -U deepeval openai -q

- source : https://docs.confident-ai.com/docs/getting-started

## overview

In [3]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [46]:
from deepeval import assert_test
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric

In [9]:
input_question = """
Berikut ini dampak yang terjadi ketika pemerintah menerapkan lump sum tax pada pasar monopoli adalah ….
A. kurva AC dan MC bergeser ke atas
B. kurva MC ke atas, sedangkan AC tetap
C. kurva VC bergeser ke atas
D. kurva MR bergeser ke kiri
E. kurva AC bergeser ke atas, sedangkan MC tetap
"""

In [10]:
output = """
Penjelasan:
Ketika pemerintah menerapkan lump sum tax pada pasar monopoli, kita perlu memahami bagaimana pajak ini mempengaruhi struktur biaya perusahaan. Lump sum tax adalah pajak yang jumlahnya tetap dan tidak bergantung pada jumlah output yang diproduksi oleh perusahaan. Ini berarti bahwa lump sum tax merupakan biaya tetap.

1. Biaya tetap tidak mempengaruhi biaya marginal (MC) karena MC hanya dipengaruhi oleh biaya yang berubah-ubah dengan produksi. Jadi, kurva MC tidak akan bergeser karena adanya lump sum tax.
2. Namun, karena lump sum tax menambah total biaya tetap, ini akan meningkatkan biaya rata-rata (AC) per unit. Oleh karena itu, kurva AC akan bergeser ke atas.

Dari penjelasan ini, kita bisa melihat bahwa jawaban yang paling tepat adalah E, yaitu kurva AC bergeser ke atas, sedangkan MC tetap. Apakah penjelasan ini membantu kamu memahami dampak lump sum tax pada struktur biaya di pasar monopoli? Bagaimana menurutmu, apakah ada bagian yang masih kurang jelas?
"""

In [16]:
output_2 = """
Penjelasan:
Ketika pemerintah menerapkan lump sum tax pada pasar monopoli, kita perlu memahami bagaimana pajak ini mempengaruhi struktur biaya perusahaan. Lump sum tax adalah pajak yang jumlahnya tetap dan tidak bergantung pada jumlah output yang diproduksi oleh perusahaan. Ini berarti bahwa lump sum tax merupakan biaya tetap.

1. Biaya tetap tidak mempengaruhi biaya marginal (MC) karena MC hanya dipengaruhi oleh biaya yang berubah-ubah dengan produksi. Jadi, kurva MC akan bergeser ke atas.
2. Namun, karena lump sum tax menambah total biaya tetap, ini akan meningkatkan biaya rata-rata (AC) per unit. Oleh karena itu, kurva AC akan bergeser ke atas.

Dari penjelasan ini, kita bisa melihat bahwa jawaban yang paling tepat adalah A, yaitu kurva AC bergeser ke atas, sedangkan MC juga akan bergeser ke atas. Apakah penjelasan ini membantu kamu memahami dampak lump sum tax pada struktur biaya di pasar monopoli? Bagaimana menurutmu, apakah ada bagian yang masih kurang jelas?
"""

In [19]:
relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
test_case = LLMTestCase(
    input=input,
    actual_output=output
)

In [20]:
test_case_2 = LLMTestCase(
    input=input,
    actual_output=output_2
)

In [21]:
relevancy_metric.measure(test_case)
print(relevancy_metric.score, relevancy_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0 The score is 1.00 because the actual output is perfectly relevant to the input. Great job! Keep up the excellent work!


In [22]:
relevancy_metric.measure(test_case_2)
print(relevancy_metric.score, relevancy_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0.7272727272727273 The score is 0.73 because the output contains several irrelevant statements, such as references to economic concepts and follow-up questions, which do not pertain to the input, a method signature or code reference. However, the score is not lower because the majority of the response still addresses the input contextually.


- wow bisa nge detect kalau jawaban dari GPT nya kurang tepat
- kalau jawaban tepat pakai `output` nilainya 1, kalau jawabanya kurang tepat pakai `output_2` score nya jadi 0.63

## metrics
- ada 14++ metrics + paper riset nya
- terbagi ke dalam 2 yaitu default metrics seperti diatas (answer relevancy metrics) dan custom metrics (GEval)

In [14]:
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval

In [26]:
output_mt = """
Pembahasan
Lump sum tax adalah jenis pajak yang tidak dipengaruhi oleh output yang diproduksi. Hal tersebut menyebabkan lump sum tax dikategorikan sebagai biaya tetap pada suatu perusahaan. Kenaikan biaya tetap menyebabkan kurva AC bergeser ke atas, namun tidak memengaruhi kurva MC sehingga kurva MC tetap. Pengaruh lump sum tax terhadap keseimbangan pasar monopoli dapat dilihat pada kurva berikut.

Jadi, jawaban yang tepat adalah E.

"""

In [25]:
#custom metrics

test_case = LLMTestCase(input=input, actual_output=output_2, expected_output=output)
correctness_metric = GEval(
    name="Correctness",
    criteria="Correctness - determine if the actual output is correct according to the expected output.",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

correctness_metric.measure(test_case)
print(correctness_metric.score, correctness_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0.44517005509863106 The actual output contains an error regarding the shift of the MC curve and an incorrect final answer (A instead of E). The rest of the text matches the expected output.


In [27]:
#custom metrics
test_case = LLMTestCase(input=input, actual_output=output, expected_output=output_mt)
correctness_metric = GEval(
    name="Correctness",
    criteria="Correctness - determine if the actual output is correct according to the expected output.",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

correctness_metric.measure(test_case)
print(correctness_metric.score, correctness_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0.7784009380298282 The actual output provides a detailed explanation similar to the expected output, but includes additional information and queries the reader, which is not present in the expected output.


- wow bisa nge detect kalau jawaban dari GPT nya kurang tepat
- komparasi jawaban output dan output_2 nilainya 0.44 karena memang output_2 salah
- sedangkan kalau komparasi jawaban output dengan output_mt nilainya 0.77 karena memang sesuai

## multiple test case

In [6]:
input_eko_5 = """
Rendi sering bertransaksi di koperasi sekolah, sedangkan Hani jarang bertransaksi di koperasi sekolah.
SHU yang didapatkan Rendi lebih besar dibandingkan Rani. Ilustrasi tersebut menunjukkan salah satu
prinsip koperasi berupa ....
A. Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing
anggota
B. Pemberian balas jasa yang terbatas terhadap modal
C. Kemandirian
D. Pendidikan perkoperasian
E. pengelolaan dilakukan secara
"""

output_eko_5 = """
Penjelasan:
Dari ilustrasi yang kamu berikan, Rendi yang sering bertransaksi di koperasi sekolah mendapatkan SHU (Sisa Hasil Usaha) yang lebih besar dibandingkan dengan Hani yang jarang bertransaksi. Ini menunjukkan bahwa jumlah SHU yang diterima oleh anggota koperasi dipengaruhi oleh seberapa sering dan seberapa besar transaksi yang dilakukan oleh anggota tersebut di koperasi.

Prinsip koperasi yang terkait dengan situasi ini adalah "Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing anggota". Artinya, setiap anggota mendapatkan bagian dari SHU berdasarkan kontribusi atau partisipasi mereka dalam kegiatan koperasi. Jadi, semakin aktif seorang anggota dalam bertransaksi, semakin besar pula SHU yang bisa ia dapatkan.

Kesimpulan:
Jawaban yang paling tepat untuk pertanyaan ini adalah A. Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing anggota. Apakah penjelasan ini membantu kamu memahami prinsip koperasi yang dimaksud? Semoga membantu, ya 😊
"""

expected_output_eko_5 = """
PEMBAHASAN :
Besarnya SHU yang diterima setiap anggota akan berbeda. Semakin banyak transaksi yang dilakukan
anggota maka kemungkinan SHU-nya semakin besar, begitu juga sebaliknya. Hal tersebut sesuai dengan
salah satu prinsip koperasi yaitu pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa
usaha masing-masing anggota.
Jadi, jawaban yang tepat adalah poin A.

"""

In [7]:
input_bing_2 = """
Nina: What do you do in the afternoon?
Karim: (18) ____ It’s my hobby.
Nina: How about your sister? (19) ____
Karim: She (20) ____ lunch with my parents. Cooking is her hobby.

The correct answer to fill in the blank number (20) is ____
A. Prepares
B. Prepare
C. Eats
D. eat
"""

output_bing_2 = """
Penjelasan:
Mari kita analisis dialog yang diberikan untuk menemukan jawaban yang tepat untuk mengisi bagian yang kosong nomor (20). Dalam dialog tersebut, Nina bertanya kepada Karim tentang apa yang dilakukan saudara perempuannya. Karim menjawab bahwa saudara perempuannya "____ lunch with my parents. Cooking is her hobby."

Dari konteks kalimat, kita tahu bahwa subjek dari kalimat adalah 'She' yang merujuk pada saudara perempuan Karim. Dalam bahasa Inggris, ketika subjeknya adalah 'She', maka kata kerja yang mengikuti harus dalam bentuk singular (tunggal). Mari kita lihat pilihan jawabannya:
A. Prepares - bentuk ketiga orang tunggal dari 'prepare'
B. Prepare - bentuk dasar, biasanya digunakan untuk plural atau 'I/you/we/they'
C. Eats - bentuk ketiga orang tunggal dari 'eat'
D. eat - bentuk dasar, biasanya digunakan untuk plural atau 'I/you/we/they'

Dari pilihan di atas, kita perlu memilih kata kerja yang sesuai dengan subjek 'She' dan konteks kalimat yang mengindikasikan kegiatan yang dilakukan saudara perempuan Karim bersama orang tua mereka. Kata 'prepares' dan 'eats' keduanya adalah bentuk ketiga orang tunggal, namun konteks kalimat lebih mengarah kepada aktivitas makan bersama, bukan mempersiapkan. Oleh karena itu, pilihan yang paling tepat adalah 'eats'.

Kesimpulan:
Jawaban yang benar untuk mengisi bagian yang kosong nomor (20) adalah C. Eats. Apakah penjelasan ini membantu kamu memahami mengapa jawaban tersebut yang paling tepat? 😊
"""

expected_output_bing_2 = """
Pada dialog (20), Karim menjawab ‘Ia ____ makan malam bersama orang tuaku. Memasak adalah hobinya.’ Karena hobi saudara perempuan Karim adalah memasak, dapat disimpulkan bahwa ia menyiapkan makan malam bersama orang tuanya atau prepare.
Untuk menyatakan kebiasaan, digunakan struktur kalimat: Subject + Verb 1 + Adverb of Time. (Catatan: Kata kerjanya harus menunjukkan suatu aktivitas).
Karena subjek kalimat adalah ‘ia (perempuan)’, maka subjeknya tunggal. Oleh karena itu, kata kerjanya mendapat akhiran +s atau +es: prepares (pilihan A).
Pilihan eat/s artinya ‘makan’. Pilihan ini tidak tepat karena tidak sesuai dengan informasi pada teks tentang hobi memasak saudara laki-laki karim.
Jadi, pilihan jawaban yang tepat adalah A.
"""

In [36]:
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset

first_test_case = LLMTestCase(input=input_eko_5, actual_output=output_eko_5, expected_output=expected_output_eko_5)
second_test_case = LLMTestCase(input=input_bing_2, actual_output=output_bing_2, expected_output=expected_output_bing_2)

dataset = EvaluationDataset(test_cases=[first_test_case, second_test_case])

In [33]:
from deepeval import evaluate

evaluate(dataset, [relevancy_metric])

Output()

Output()

/usr/local/lib/python3.10/dist-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the response is perfectly relevant and directly addresses the input question with no irrelevant statements. Great job!, error: None)

For test case:

  - input: 
Rendi sering bertransaksi di koperasi sekolah, sedangkan Hani jarang bertransaksi di koperasi sekolah.
SHU yang didapatkan Rendi lebih besar dibandingkan Rani. Ilustrasi tersebut menunjukkan salah satu
prinsip koperasi berupa ....
A. Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing
anggota
B. Pemberian balas jasa yang terbatas terhadap modal
C. Kemandirian
D. Pendidikan perkoperasian
E. pengelolaan dilakukan secara

  - actual output: 
Penjelasan:
Dari ilustrasi yang kamu berikan, Rendi yang sering bertransaksi di koperasi sekolah mendapatkan SHU (Sisa Hasil Usaha) yang lebih besar dibandingkan dengan Hani yang jarang bertransaksi. Ini menunjukka

/usr/local/lib/python3.10/dist-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

[TestResult(success=True, metrics=[<deepeval.metrics.answer_relevancy.answer_relevancy.AnswerRelevancyMetric object at 0x79447dd866e0>], input='\nRendi sering bertransaksi di koperasi sekolah, sedangkan Hani jarang bertransaksi di koperasi sekolah.\nSHU yang didapatkan Rendi lebih besar dibandingkan Rani. Ilustrasi tersebut menunjukkan salah satu\nprinsip koperasi berupa ....\nA. Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing\nanggota\nB. Pemberian balas jasa yang terbatas terhadap modal\nC. Kemandirian\nD. Pendidikan perkoperasian\nE. pengelolaan dilakukan secara\n', actual_output='\nPenjelasan:\nDari ilustrasi yang kamu berikan, Rendi yang sering bertransaksi di koperasi sekolah mendapatkan SHU (Sisa Hasil Usaha) yang lebih besar dibandingkan dengan Hani yang jarang bertransaksi. Ini menunjukkan bahwa jumlah SHU yang diterima oleh anggota koperasi dipengaruhi oleh seberapa sering dan seberapa besar transaksi yang dilakukan oleh anggota tersebut

In [37]:
evaluate(dataset, [correctness_metric])

Output()

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Output()

/usr/local/lib/python3.10/dist-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(




Metrics Summary

  - ✅ Correctness (GEval) (score: 0.7853678891424587, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The actual output correctly describes the principle of fair SHU distribution and concludes with the correct answer as in the expected output. However, it adds extra phrases and an emoji that are not present in the expected output., error: None)

For test case:

  - input: 
Rendi sering bertransaksi di koperasi sekolah, sedangkan Hani jarang bertransaksi di koperasi sekolah.
SHU yang didapatkan Rendi lebih besar dibandingkan Rani. Ilustrasi tersebut menunjukkan salah satu
prinsip koperasi berupa ....
A. Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing
anggota
B. Pemberian balas jasa yang terbatas terhadap modal
C. Kemandirian
D. Pendidikan perkoperasian
E. pengelolaan dilakukan secara

  - actual output: 
Penjelasan:
Dari ilustrasi yang kamu berikan, Rendi yang sering bertransaksi di koperasi sekolah mendapatkan S

/usr/local/lib/python3.10/dist-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

[TestResult(success=True, metrics=[<deepeval.metrics.g_eval.g_eval.GEval object at 0x79447ef18c40>], input='\nRendi sering bertransaksi di koperasi sekolah, sedangkan Hani jarang bertransaksi di koperasi sekolah.\nSHU yang didapatkan Rendi lebih besar dibandingkan Rani. Ilustrasi tersebut menunjukkan salah satu\nprinsip koperasi berupa ....\nA. Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing\nanggota\nB. Pemberian balas jasa yang terbatas terhadap modal\nC. Kemandirian\nD. Pendidikan perkoperasian\nE. pengelolaan dilakukan secara\n', actual_output='\nPenjelasan:\nDari ilustrasi yang kamu berikan, Rendi yang sering bertransaksi di koperasi sekolah mendapatkan SHU (Sisa Hasil Usaha) yang lebih besar dibandingkan dengan Hani yang jarang bertransaksi. Ini menunjukkan bahwa jumlah SHU yang diterima oleh anggota koperasi dipengaruhi oleh seberapa sering dan seberapa besar transaksi yang dilakukan oleh anggota tersebut di koperasi.\n\nPrinsip koperasi ya

## add test case from csv

In [ ]:
from deepeval.dataset import EvaluationDataset

dataset = EvaluationDataset()
dataset.add_test_cases_from_csv_file(
    # file_path is the absolute path to you .csv file
    file_path="example.csv",
    input_col_name="query",
    actual_output_col_name="actual_output",
    expected_output_col_name="expected_output",
    context_col_name="context",
    context_col_delimiter= ";",
    retrieval_context_col_name="retrieval_context",
    retrieval_context_col_delimiter= ";"
)

## invoke openai

In [8]:
from openai import OpenAI

client = OpenAI()
prompt = """
Teach in a conversational manner and make it digestible. When possible, always end your sentence with a probing question to check student's understanding before moving on. Explain with bahasa (bahasa as default language) even though the input is non bahasa (ex: english subject) but for the material still using the original language. Please use latex format when you need any formula / equation.
The latex formula must be in the format
(1) $ example formula $
(2) $$ example formula $$
(3) \( example formula \)
(4) \[ example formula \]
NOTE: Replace the newline "\n" with whitespace (" ") for EVERY LATEX FORMULA

As a conversational AI tutor, you are expected to guide students through solving questions or issues they present. To do this effectively:

1. Analyze the question first, understanding the context, topic, and all relevant concepts. Questions may encompass multiple concepts. After breaking these concepts down, create a plan to solve the issue. Share brief explanations and/or formulas as needed to make sure you grasp these concepts, allowing you to provide a well-justified response.
2. Begin your response with the phrase ""Penjelasan: "" and proceed to provide a step-by-step explanation of your solution, making sure to use new lines or numbering for each step to enhance comprehension. Replace specific programming or coding syntax with more conceptual representations, like √ instead of ""sqrt"" and aⁿ instead of a^n, and a/b instead \frac{a}{b}. Remove any encoded latex symbol like ""\( \) or \[ \]""
3. Conclude your response with the final answer, placing it after the phrase ""Kesimpulan: "". Add a friendly note such as ""Semoga membantu, ya 😊"" to create a more personal connection with the student.
4. In answering TRUE or FALSE questions, be critical and thorough. Some statements may be designed to mislead. Explore the statement's context in detail, question your judgments, and avoid potential traps. If the truth of a statement depends on the context (meaning it's only partially true), consider it FALSE.

Remember to inspect past conversations for context, and refer to information from similar questions to aid your answer.

Please PRIORITIZE using BAHASA INDONESIA (Bahasa) as default language even when the input is in English (non-Bahasa). However, appropriate academic materials may be delivered in their original language for accurate understanding.

Remember, you will have:
access to input from prior conversations: {{$history}}

Here is the question that asked by user
user: {{$input}}

a similar question's text: {{$question_text}}
its answer: {{$answer}}
and its answer choices: {{$options}}.

REMEMBER to explain with BAHASA INDONESIA even though the input is non bahasa (ex: english subject) but for the material still using the original.
assistant:
"""

In [10]:
response = client.chat.completions.create(
  model="gpt-4o",
  temperature=0,
  presence_penalty=0,
  frequency_penalty=0,
  max_tokens=1000,
  messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": input_eko_5}
  ]
)

In [11]:
response.choices[0].message.content

'Penjelasan: \n\nUntuk menjawab pertanyaan ini, kita perlu memahami prinsip-prinsip koperasi. Salah satu prinsip koperasi adalah pembagian Sisa Hasil Usaha (SHU) yang dilakukan secara adil dan sebanding dengan besarnya jasa usaha masing-masing anggota. \n\nDalam ilustrasi yang diberikan, Rendi sering bertransaksi di koperasi sekolah, sedangkan Hani jarang bertransaksi. Akibatnya, SHU yang didapatkan Rendi lebih besar dibandingkan dengan Hani. Hal ini menunjukkan bahwa pembagian SHU dilakukan berdasarkan besarnya jasa usaha masing-masing anggota.\n\nMari kita lihat pilihan jawabannya:\nA. Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing anggota - Ini sesuai dengan ilustrasi yang diberikan.\nB. Pemberian balas jasa yang terbatas terhadap modal - Ini tidak relevan dengan ilustrasi.\nC. Kemandirian - Ini juga tidak relevan dengan ilustrasi.\nD. Pendidikan perkoperasian - Ini tidak relevan dengan ilustrasi.\nE. Pengelolaan dilakukan secara - Ini tidak l

In [12]:
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset

first_test_case = LLMTestCase(input=input_eko_5, actual_output=response.choices[0].message.content, expected_output=expected_output_eko_5)

dataset = EvaluationDataset(test_cases=[first_test_case])

In [15]:
from deepeval import evaluate

#custom metrics
correctness_metric = GEval(
    name="Correctness",
    criteria="Correctness - determine if the actual output is correct according to the expected output.",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

result = evaluate(dataset, [correctness_metric])

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Output()



Metrics Summary

  - ✅ Correctness (GEval) (score: 0.8869708498159357, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The actual output correctly identifies that SHU is distributed based on the members' business services, matching the expected output. Minor discrepancy in format and additional explanation slightly affects the score., error: None)

For test case:

  - input: 
Rendi sering bertransaksi di koperasi sekolah, sedangkan Hani jarang bertransaksi di koperasi sekolah.
SHU yang didapatkan Rendi lebih besar dibandingkan Rani. Ilustrasi tersebut menunjukkan salah satu
prinsip koperasi berupa ....
A. Pembagian SHU dilakukan secara adil sebanding dengan besarnya jasa usaha masing-masing
anggota
B. Pemberian balas jasa yang terbatas terhadap modal
C. Kemandirian
D. Pendidikan perkoperasian
E. pengelolaan dilakukan secara

  - actual output: Penjelasan: 

Untuk menjawab pertanyaan ini, kita perlu memahami prinsip-prinsip koperasi. Salah satu prinsip koperasi adalah

/usr/local/lib/python3.10/dist-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

In [18]:
result[0].success

True

In [19]:
correctness_metric.measure(first_test_case)
correctness_metric.score

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0.902012875502605

## end to end
- read gsheet from colabs (https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-part-2-b867786aed55)

### read data from gsheet

In [28]:
import pandas as pd

In [20]:
# 1. Authorizing google colab
from google.colab import auth
auth.authenticate_user()

In [21]:
# 2. credentials for google sheets
import gspread
from google.auth import default
creds, _ = default()

In [22]:
# 3. authotizing the connection
gc = gspread.authorize(creds)

In [25]:
#4. Connecting
worksheet = gc.open('eval_dataset').sheet1

In [27]:
#5. Exporting data to get_all_values gives a list of rows.
rows = worksheet.get_all_values()

In [31]:
# 6. Using pandas to convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows)

# creating columns name
df.columns = df.iloc[0]
df = df.iloc[1:]

In [32]:
df

,nomer,question,jawaban_mt
1,1,Berikut ini dampak yang terjadi ketika pemerin...,Pembahasan\nLump sum tax adalah jenis pajak ya...
2,2,Pak Tono memiliki sebidang tanah yang harga pe...,Pembahasan : \n\nTahap 1 (Menghitung besarnya ...
3,3,Perhatikan data keuangan PT Ruang Buku berikut...,Pembahasan\nBTUD = Persediaan awal + Pembelian...
4,5,"Rendi sering bertransaksi di koperasi sekolah,...",PEMBAHASAN :\nBesarnya SHU yang diterima setia...
5,6,Diketahui suatu negara mempunyai data perekono...,Diketahui:\nJumlah konsumsi 200.000\nPenerimaa...
6,7,"Negara A memiliki nilai indeks gini 0,43, seda...",PEMBAHASAN :\nRasio indeks gini berkisar antar...
7,10,Berikut ini pernyataan yang tepat mengenai teo...,Kunci Jawaban : D\nPembahasan :\nSchumpeter me...


In [33]:
# 4. Importing Google Sheet using google sheet key
eval_dataset_gsheet = gc.open_by_key('1YXM_hlkfe6PeJm3lT9Zdw1BsVGwibELZSRVOvk8muLg')

# 5. Bring the tab you want to update
test_googlept2 = test_google.worksheet('ekonomi')

### invoke openai

In [40]:
test_googlept2.update_cell(1, 4, "jawaban_gpt")

{'spreadsheetId': '1YXM_hlkfe6PeJm3lT9Zdw1BsVGwibELZSRVOvk8muLg',
 'updatedRange': 'ekonomi!D1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [47]:
test_googlept2.update_cell(1, 5, "correctness_score")

{'spreadsheetId': '1YXM_hlkfe6PeJm3lT9Zdw1BsVGwibELZSRVOvk8muLg',
 'updatedRange': 'ekonomi!E1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [43]:
def call_openai(prompt, user_query):

  try:
    response = client.chat.completions.create(
      model="gpt-4o",
      temperature=0,
      presence_penalty=0,
      frequency_penalty=0,
      max_tokens=1000,
      messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": user_query}
      ]
    )

    return response.choices[0].message.content
  except:
    return "error"

In [45]:
for i, question in enumerate(df['question']):

  answer_gpt = call_openai(prompt, question)
  test_googlept2.update_cell(i+2, 4, answer_gpt)

In [51]:
worksheet = gc.open('eval_dataset').sheet1
rows = worksheet.get_all_values()
df2 = pd.DataFrame.from_records(rows)
df2.columns = df2.iloc[0]
df2 = df2.iloc[1:]

In [52]:
df2

,nomer,question,jawaban_mt,jawaban_gpt,correctness_score
1,1,Berikut ini dampak yang terjadi ketika pemerin...,Pembahasan\nLump sum tax adalah jenis pajak ya...,Penjelasan: \nUntuk memahami dampak dari pener...,
2,2,Pak Tono memiliki sebidang tanah yang harga pe...,Pembahasan : \n\nTahap 1 (Menghitung besarnya ...,Penjelasan: Untuk menentukan luas tanah yang d...,
3,3,Perhatikan data keuangan PT Ruang Buku berikut...,Pembahasan\nBTUD = Persediaan awal + Pembelian...,Penjelasan: Untuk menghitung besarnya barang y...,
4,5,"Rendi sering bertransaksi di koperasi sekolah,...",PEMBAHASAN :\nBesarnya SHU yang diterima setia...,"Penjelasan: \n\nUntuk menjawab pertanyaan ini,...",
5,6,Diketahui suatu negara mempunyai data perekono...,Diketahui:\nJumlah konsumsi 200.000\nPenerimaa...,Penjelasan: Untuk menghitung pendapatan per ka...,
6,7,"Negara A memiliki nilai indeks gini 0,43, seda...",PEMBAHASAN :\nRasio indeks gini berkisar antar...,Penjelasan: \nIndeks Gini adalah ukuran ketimp...,
7,10,Berikut ini pernyataan yang tepat mengenai teo...,Kunci Jawaban : D\nPembahasan :\nSchumpeter me...,Penjelasan: Teori pertumbuhan ekonomi dari Jos...,


In [54]:
for i in range(len(df2)):
  input_question = df2['question'][i+1]
  actual_output = df2['jawaban_gpt'][i+1]
  expected_output = df2['jawaban_mt'][i+1]

  test_case = LLMTestCase(input=input_question,
                          actual_output=actual_output,
                          expected_output=expected_output)

  correctness_metric.measure(test_case)
  score = correctness_metric.score
  test_googlept2.update_cell(i+2, 5, score)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...